# Similar to the code in "HydroShed_MeshGenerator.ipynb" to generate a mesh

In [ ]:
# Libraries

import numpy as np
from skimage.transform import resize
import pymartini
import rasterio
import pyvista as pv
from scipy.ndimage import map_coordinates
import settings
import utils

# matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.ticker as ticker
# for interactions
from mpl_toolkits.mplot3d import Axes3D

import TIN_engine
from TIN_engine import *
from TIN_draw import *
from TIN_drainage import *
from TIN_watershed import *
from TIN import *

# remove this (and possibly restart the kernel) if you don't want interactive plots
%matplotlib widget

# for reloading modules (specifically TIN_engine) during development
%load_ext autoreload
%autoreload 2

In [ ]:
# ---- Load DEM and create TIN mesh ----
z_scale = 0.01
vertices_3d, triangles, xs, ys, zs, zs_scaled  = get_triangles_from_DEM(settings.WASHINGTON_SMALL, mesh_level=10, z_scale=z_scale)
mesh = get_mesh_from_triangles(triangles, vertices_3d)

In [ ]:
# Plot the mesh
pv.plot(mesh, jupyter_backend='static')

In [ ]:
# plot some of the verticies

# only plotting a subset here
num = 10000
X = xs[0:num] 
Y = ys[0:num]
Z = zs[0:num]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X, Y, Z, s=0.5, c=Z, marker='o', cmap='viridis')

plt.show()

In [ ]:
# get highest point
max_index = np.nanargmax(zs)
highest_point = [xs[max_index], ys[max_index], zs[max_index]]
highest_point

In [ ]:
#triangles_subset = get_subset_of_triangles_from_bounds(triangles, [-116.5, 49.34, -116.52, 49.32], xs, ys)
radius = 0.05
bounds_around_highest = [highest_point[0] + radius, highest_point[1] + radius, highest_point[0] - radius, highest_point[1] - radius]

triangles_subset = get_subset_of_triangles_from_bounds(triangles, bounds_around_highest, xs, ys)

In [ ]:
triangle_objects, vertices = convert_to_triangle_and_vertex_objects(triangles_subset, xs, ys, zs)
print(len(triangle_objects), "triangle objects created.")
print(len(vertices), "vertex objects created.")

In [ ]:
lines = []

points = calculate_steepest_descent_line(highest_point, triangle_objects, triangles_subset, xs, ys, zs)
lines.append(points)

In [ ]:
has_flat_triangles(triangle_objects)
flat = get_flat_triangles(triangle_objects)
len(flat)
unflaten_triangles(triangle_objects)

In [ ]:
# ---- Drainage network calculation ----
drainage_outlet_nodes = create_drainage_network(triangle_objects)
print(len(drainage_outlet_nodes), "outlet nodes created.")

In [ ]:
# Visualize drainage network

# draw triangles
fig = plt.figure()
ax = fig.add_subplot(111)
for triangle in triangles_subset:
    draw_triangle(ax, triangle, "#00000055", xs, ys)

print("Finished drawing triangles.")

network_lines = []

def draw_node(node: Node, depth, color):
    if depth > 400:
        print("Max depth reached, stopping recursion.")
        return

    for upstream_node in node.upstream_nodes:
        draw_line_points(ax, node.point[0:2], upstream_node.point[0:2], color, linewidth=2)
        network_lines.append([node.point, upstream_node.point])
        draw_node(upstream_node, depth + 1, color)

for outlet in drainage_outlet_nodes:
    draw_point(ax, outlet.point[0:2], markersize=4)
    random_color = np.random.rand(3,)

    # ensure not too light
    random_color = random_color * 0.7

    draw_node(outlet, 0, random_color)

print("Done drawing network.")

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Drainage Networks", size=14)
plt.show()

In [ ]:
# ---- Watershed delineation ----
watershed, start_node = delineate_random_watershed(drainage_outlet_nodes)

print(len(watershed), "triangles")

In [ ]:
# ---- Visualize watershed ----

SHOW_DRAINAGE_NETWORK = True

# draw triangles
fig = plt.figure()
ax = fig.add_subplot(111)
for triangle in triangles_subset:
    draw_triangle(ax, triangle, "#00000055", xs, ys)

print("Finished drawing terrain triangles.")

for triangle in watershed:
    draw_triangle_object(ax, triangle, "#00ff0055", filled=True)

print("Done drawing watershed triangles.")

if SHOW_DRAINAGE_NETWORK:
    draw_point(ax, start_node.point[0:2], markersize=4)
    draw_node(start_node, 0, (0.0, 0.0, 1.0))

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Delineated Watershed", size=14)
plt.show()

In [ ]:
# ---- 3D Visualization of network with PyVista ----
ACTIVE = False

if ACTIVE:
    def show_lines(plotter: pv.Plotter):
        # draw lines
        z_offset = 0.0001
        previous_percentange = 0

        i = 0
        for line in network_lines:
            prev_point = line[0]

            new_line = []
            p2 = None
            for point in line[1:]:
                p1 = [prev_point[0], prev_point[1], (prev_point[2] * z_scale) + z_offset]
                p2 = [point[0], point[1], (point[2] * z_scale) + z_offset]
                new_line.append(p1)
                new_line.append(p2)
                prev_point = point

            plotter.add_lines(np.array(new_line), color='blue', width=5)

            i += 1
            current_percentage = int((i / len(network_lines)) * 100)
            if current_percentage != previous_percentange:
                print(f"Progress: {current_percentage}%")
                previous_percentange = current_percentage

        print("Done drawing lines.")
        

    p = pv.Plotter()
    p.add_mesh(get_mesh_from_triangles(triangles_subset, vertices_3d), show_edges=False, color='lightgray')
    show_lines(p)
    p.show() # fully interactive

In [ ]:
# steepest descent lines
lines = []

# claculate descent lines
for triangle in triangle_objects: 
    start_point = triangle.get_centroid().coord()
    points = calculate_steepest_descent_line(start_point, triangle_objects, triangles_subset, xs, ys, zs)
    lines.append(points)

In [ ]:
# draw triangles
fig = plt.figure()
ax = fig.add_subplot(111)
for triangle in triangles_subset:
    draw_triangle(ax, triangle, "#00000055", xs, ys)

print("Finished drawing triangles.")

# draw lines
for line in lines:
    thickness = 1
    thickness_growth = 0.1
    color = (0, 0, 1)
    color_growth = (0.05, 0, 0)

    prev_point = line[0]
    for point in line[1:]:
        draw_line_points(ax, prev_point, point, "blue", linewidth=1)
        color = (min(color[0] + color_growth[0], 1), min(color[1] + color_growth[1], 1), min(color[2] + color_growth[2], 1))
        thickness += thickness_growth
        prev_point = point

print("Done drawing lines.")

plt.show()

In [ ]:
ACTIVE = False

if ACTIVE:
    p = pv.Plotter()
    p.add_mesh(get_mesh_from_triangles(triangles_subset, vertices_3d), show_edges=False, color='lightgray')

    z_offset = 0.0005
    previous_percentange = 0

    i = 0
    for line in lines:
        prev_point = line[0]

        new_line = []
        p2 = None
        for point in line[1:]:
            p1 = [prev_point[0], prev_point[1], (prev_point[2] * z_scale) + z_offset]
            p2 = [point[0], point[1], (point[2] * z_scale) + z_offset]
            new_line.append(p1)
            new_line.append(p2)
            prev_point = point

        #p.add_lines(np.array(new_line), color='blue', width=5)

        i += 1
        current_percentage = int((i / len(lines)) * 100)
        if current_percentage != previous_percentange:
            print(f"Progress: {current_percentage}%")
            previous_percentange = current_percentage

    print("Done drawing lines.")

    #p.show(jupyter_backend='static')
    #p.show(jupyter_backend='html') # simple interactive
    p.show() # fully interactive

In [ ]:
# a mask just for some small section of the data set
mask = (xs < -116.5) & (xs > -116.54) & (ys < 49.34) & (ys > 49.30)

verticies_subset = np.where(mask)
m = np.isin(triangles, verticies_subset)
triangles_subset = triangles[np.all(m, axis=1)]

In [ ]:
# Draws a nice graph that shows the lines of steepest descent from each triangle's centroid (see similar in Johnes. et al., p. 1241)
# This is old code, see `steepest_descent_line` for better and faster implementation

# plot some vertices in 2D
X = xs[mask]
Y = ys[mask]
Z = zs[mask]

fig = plt.figure()
ax = fig.add_subplot(111)

POINT_MARKER_SIZE = 1

triangle_objects, vers = convert_to_triangle_and_vertex_objects(triangles_subset, xs, ys, zs)

# draw triangles
for triangle in triangles_subset:
    draw_triangle(ax, triangle, "#00000055", xs, ys)

for a_triangle in triangle_objects:
    thickness = 1
    thickness_growth = 0.1
    start_point = a_triangle.get_centroid().coord()[0:2]

    tri = get_triangle_at(start_point, triangles_subset, xs, ys, zs)

    s_point = start_point
    for i in range(1, 100):
        full_tri = get_full_3D_triangle(tri, xs, ys, zs)

        descent = calculate_steepest_descent(full_tri)
        descent = descent / np.linalg.norm(descent)
        descent *= 0.001

        next_point, adj_tri, v1, v2 = get_point_and_adj_triangle_from_descent(tri, s_point, descent, xs, ys, zs, triangles_subset)
        if next_point is None:
            print("next_point should not be None, stopping at iteration ", i)
            break

        draw_line_points(ax, s_point, next_point, "blue", linewidth=thickness)
        thickness += thickness_growth
        if adj_tri is None:
            print("No adjacent triangle found, stopping at iteration ", i)
            break

        full_adj_tri = get_full_3D_triangle(adj_tri, xs, ys, zs)
        descent_adj = calculate_steepest_descent(full_adj_tri) # gh from p. 1239, Jones et al.

        full_adj_tri = make_triangle_counterclockwise(full_adj_tri)

        coord1 = get_real_vertex_3D(v1, xs, ys, zs)
        coord2 = get_real_vertex_3D(v2, xs, ys, zs)
        v1 = Vertex(coord1[0], coord1[1], coord1[2], v1)
        v2 = Vertex(coord2[0], coord2[1], coord2[2], v2)

        if (find_row_index(full_adj_tri, v1.coord()) + 1) % 3 == find_row_index(full_adj_tri, v2.coord()): # if v1 comes before v2 
            ij = v2.coord() - v1.coord()
        else: # v2 comes before v1
            # swap
            temp = v1
            v1 = v2
            v2 = temp

            ij = v2.coord() - v1.coord()

        # direction of the adj. triangle: if positive then adj. triangle slopes toward the current edge
        direction = utils.cross_2D(np.array(descent_adj), ij[0:2])
        
        if direction < 0:
            tri = adj_tri
            s_point = next_point
            continue
        else:
            lowest = None
            if v1.z > v2.z:
                lowest = v2
            else:
                lowest = v1
            
            draw_line_points(ax, next_point, lowest.coord()[0:2], "blue", linewidth=thickness)
            thickness += thickness_growth


            stop = False
            while True:
                ordered = get_all_triangles_and_edges_at_point(lowest, triangle_objects)
                color = (1, 0, 0)
                previous_item = None
                next_item = None
                j = 0
                next_point = None
                next_triangle = None
                for item in ordered:
                    previous_item = ordered[(j - 1) % len(ordered)]
                    next_item = ordered[(j + 1) % len(ordered)]

                    if isinstance(item, Triangle):
                        if test_triangle(lowest, item):
                            next_triangle = item
                    else: # must be an edge
                        # next_item and previous_item should be triangles adjacent to this edge
                        if test_edge(lowest, item, next_item, previous_item):
                            draw_line(ax, item, "blue", linewidth=thickness)
                            thickness += thickness_growth
                            next_point = get_other_vertex_from_edge(lowest, item)
                    
                    color = ((color[0] - (1.0 / 30.0)) % 1.0, color[1], color[2])
                    j += 1
                
                if next_point is not None:
                    lowest = next_point
                    continue
                elif next_triangle is not None:
                    tri = next_triangle.convert_to_indices()
                    s_point = lowest.coord()[0:2]
                    break # and then continue the outer for loop
                else:
                    # need to stop here
                    stop = True
                    break

            
            if stop:
                break
            else:
                continue

ax.xaxis.set_major_locator(ticker.MaxNLocator(5)) # to prevent crowding on the x-axis
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Lines of steepest descent across TIN", size=14)
plt.show()